<a href="https://colab.research.google.com/github/Abdul-Haseeb-boonig/CS-417-Lab9/blob/main/ParallelProcessingLab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 16.8 MB/s eta 0:00:00


In [12]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [13]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xeff5m_r
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xeff5m_r
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=1704fff7a33ec9e696ff9210d5ec4bdb6587a7e0a1a3d16e03749f07f8b81e6b
  Stored in directory: /tmp/pip-ephem-wheel-cache-r5e81bfh/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [14]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [20]:
%%cu
#include <iostream>
#include <cuda_runtime.h>
// Matrix multiplication kernel
__global__ void matrixMul(const int *A, const int *B, int *C, int N) {
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
if (row < N && col < N) {
int result = 0;
for (int k = 0; k < N; ++k) {
result += A[row * N + k] * B[k * N + col];
}
C[row * N + col] = result;
}
}
// Function to print a matrix
void printMatrix(const int *matrix, int rows, int cols) {
for (int i = 0; i < rows; ++i) {
for (int j = 0; j < cols; ++j) {
std::cout << matrix[i * cols + j] << " ";
}
std::cout << std::endl;
}
std::cout << std::endl;
}
int main() {
const int N = 3; // Size of the matrices (NxN)
// Example matrices A and B
const int A[N][N] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
const int B[N][N] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
// Allocate memory for matrices on the host
int C_cpu[N][N]; // Result matrix from CPU
int C_gpu[N][N]; // Result matrix from GPU
// Allocate memory for matrices on the device
int *d_A, *d_B, *d_C;
cudaMalloc((void **)&d_A, N * N * sizeof(int));
cudaMalloc((void **)&d_B, N * N * sizeof(int));
cudaMalloc((void **)&d_C, N * N * sizeof(int));
// Copy matrices A and B from host to device
cudaMemcpy(d_A, &A[0][0], N * N * sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(d_B, &B[0][0], N * N * sizeof(int), cudaMemcpyHostToDevice);
// Define block and grid dimensions
dim3 blockDim(2, 2); // 2x2 thread block
dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);
// Launch the matrix multiplication kernel
matrixMul<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);
// Copy the result matrix from device to host
cudaMemcpy(&C_gpu[0][0], d_C, N * N * sizeof(int), cudaMemcpyDeviceToHost);
// Matrix multiplication on CPU for comparison
for (int i = 0; i < N; ++i) {
for (int j = 0; j < N; ++j) {
C_cpu[i][j] = 0;
for (int k = 0; k < N; ++k) {
C_cpu[i][j] += A[i][k] * B[k][j];
}
}
}
// Print matrices A, B, and C_cpu (result from CPU)
std::cout << "Matrix A:" << std::endl;
printMatrix(&A[0][0], N, N);
std::cout << "Matrix B:" << std::endl;
printMatrix(&B[0][0], N, N);
std::cout << "Result from CPU (C_cpu):" << std::endl;
printMatrix(&C_cpu[0][0], N, N);
// Print the result from the GPU (C_gpu)
std::cout << "Result from GPU (C_gpu):" << std::endl;
printMatrix(&C_gpu[0][0], N, N);
// Compare the results
bool resultMatch = true;
for (int i = 0; i < N; ++i) {
for (int j = 0; j < N; ++j) {
if (C_cpu[i][j] != C_gpu[i][j]) {
resultMatch = false;
break;
}
}
}
if (resultMatch) {
std::cout << "Results match between CPU and GPU implementations." << std::endl;
} else {
std::cout << "Results do not match between CPU and GPU implementations." << std::endl;
}
// Free device memory
cudaFree(d_A);
cudaFree(d_B);
cudaFree(d_C);
return 0;
}

Matrix A:
1 2 3 
4 5 6 
7 8 9 

Matrix B:
9 8 7 
6 5 4 
3 2 1 

Result from CPU (C_cpu):
30 24 18 
84 69 54 
138 114 90 

Result from GPU (C_gpu):
0 0 0 
0 0 0 
0 0 0 

Results do not match between CPU and GPU implementations.



In [35]:
%%cu
#include <iostream>
#include <iomanip>
#include <cuda_runtime.h>
__global__ void matrixMul(const int *A, const int *B, int *C, int N) {
    // Define shared memory for tiles of matrices A and B
    __shared__ int tileA[2][2];
    __shared__ int tileB[2][2];

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int result = 0;

    // Loop over tiles
    for (int i = 0; i < N; i += blockDim.x) {
        // Load tiles into shared memory
        tileA[threadIdx.y][threadIdx.x] = A[row * N + i + threadIdx.x];
        tileB[threadIdx.y][threadIdx.x] = B[(i + threadIdx.y) * N + col];

        // Synchronize to make sure tiles are loaded before computation
        __syncthreads();

        // Perform the matrix multiplication using the loaded tiles
        for (int k = 0; k < blockDim.x; ++k) {
            result += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }

        // Synchronize before loading the next tile
        __syncthreads();
    }

    // Write the result to the output matrix
    if (row < N && col < N) {
        C[row * N + col] = result;
    }
}
// Function to print a matrix
void printMatrix(const int *matrix, int rows, int cols) {
for (int i = 0; i < rows; ++i) {
for (int j = 0; j < cols; ++j) {
std::cout << matrix[i * cols + j] << " ";
}
std::cout << std::endl;
}
std::cout << std::endl;
}
int main() {
const int N = 3; // Size of the matrices (NxN)
// Example matrices A and B
const int A[N][N] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
const int B[N][N] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
// Allocate memory for matrices on the host
int C_cpu[N][N]; // Result matrix from CPU
int C_gpu[N][N]; // Result matrix from GPU
// Allocate memory for matrices on the device
int *d_A, *d_B, *d_C;
cudaMalloc((void **)&d_A, N * N * sizeof(int));
cudaMalloc((void **)&d_B, N * N * sizeof(int));
cudaMalloc((void **)&d_C, N * N * sizeof(int));
// Copy matrices A and B from host to device
cudaMemcpy(d_A, &A[0][0], N * N * sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(d_B, &B[0][0], N * N * sizeof(int), cudaMemcpyHostToDevice);
// Define block and grid dimensions
dim3 blockDim(2, 2); // 2x2 thread block
dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);
clock_t t;
t = clock();
// Launch the matrix multiplication kernel
matrixMul<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);
t = clock() - t;
double time_taken = ((double)t)/CLOCKS_PER_SEC; // in seconds
// Copy the result matrix from device to host
cudaMemcpy(&C_gpu[0][0], d_C, N * N * sizeof(int), cudaMemcpyDeviceToHost);
// Matrix multiplication on CPU for comparison
for (int i = 0; i < N; ++i) {
for (int j = 0; j < N; ++j) {
C_cpu[i][j] = 0;
for (int k = 0; k < N; ++k) {
C_cpu[i][j] += A[i][k] * B[k][j];
}
}
}
// Print the result from the GPU (C_gpu)
std::cout << "Result from GPU (C_gpu):" << std::endl;
printMatrix(&C_gpu[0][0], N, N);

// Compare the results
bool resultMatch = true;
for (int i = 0; i < N; ++i) {
for (int j = 0; j < N; ++j) {
if (C_cpu[i][j] != C_gpu[i][j]) {
resultMatch = false;
break;
}
}
}
if (resultMatch) {
std::cout << "Results match between CPU and GPU implementations." << std::endl;
} else {
std::cout << "Results do not match between CPU and GPU implementations." << std::endl;
}
printf("GPU with Global Memory took %f seconds to execute \n", time_taken);
// Free device memory
cudaFree(d_A);
cudaFree(d_B);
cudaFree(d_C);
return 0;
}

Result from GPU (C_gpu):
0 0 0 
0 0 0 
0 0 0 

Results do not match between CPU and GPU implementations.
GPU with Global Memory took 0.000006 seconds to execute 

